# LangGraph Text Summarization Agent
A comprehensive text summarization system using LangGraph with multiple reflection steps.

In [2]:
# Install required packages
! pip install langgraph pyboxen openai

DEPRECATION: Loading egg at c:\users\sachi\appdata\local\programs\python\python311\lib\site-packages\scrape_linkedin-0.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
from typing import Dict, List, Tuple
from langgraph.graph import Graph
import pyboxen
from openai import OpenAI
import time

In [6]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [7]:
# Initialize OpenAI client
client = OpenAI()

In [20]:
class SummarizationAgent:
    def __init__(self):
        self.files_content = {}
        self.individual_summaries = {}
        self.final_summary = ""
        
    def print_step(self, message: str):
        """Print a step indicator"""
        print(f"\n{'='*80}\n{message}\n{'='*80}")
        
    def print_box(self, title: str, content: str, style: str = "blue"):
        """Helper function to print formatted boxes using pyboxen"""
        print(boxen(
            content,
            title=title,
            color=style,
            padding=1,
            margin=1
        ))

    def read_files(self, folder_path: str = "Files") -> None:
        """Read all text files from the specified folder"""
        self.print_step("📂 PHASE 1: READING FILES FROM DIRECTORY")
        
        for filename in os.listdir(folder_path):
            if filename.endswith('.txt'):
                print(f"\n→ Processing file: {filename}")
                with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                    content = file.read()
                    self.files_content[filename] = content
                    self.print_box(
                        f"📄 File Content: {filename}",
                        content,
                        "green"
                    )
                print(f"✓ Successfully read: {filename}")
                time.sleep(1)  # Brief pause for readability

    def reflect_and_summarize(self, text: str, filename: str) -> str:
        """Perform iterative summarization with reflections"""
        print(f"\n→ Starting analysis for: {filename}")
        
        # First Pass - Initial Analysis
        print("\n⚡ REFLECTION PASS 1: Initial Analysis")
        messages = [
            {"role": "system", "content": "You are a skilled document summarizer. First, analyze the main themes and key points."},
            {"role": "user", "content": f"Analyze and summarize the key points from this text: {text}"}
        ]
        
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.7
        )
        initial_analysis = response.choices[0].message.content
        
        self.print_box(
            "📝 Initial Analysis Results",
            f"Key Points Identified:\n{initial_analysis}",
            "blue"
        )
        
        # Second Pass - Refinement
        print("\n⚡ REFLECTION PASS 2: Refinement")
        messages.append({"role": "assistant", "content": initial_analysis})
        messages.append({
            "role": "user", 
            "content": "Based on this analysis, create a more structured and concise summary focusing on the most impactful points:"
        })
        
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.7
        )
        refined_summary = response.choices[0].message.content
        
        self.print_box(
            "🔄 Refined Analysis",
            f"Structured Summary:\n{refined_summary}",
            "yellow"
        )
        
        # Third Pass - Final Polish
        print("\n⚡ REFLECTION PASS 3: Final Polish")
        messages.append({"role": "assistant", "content": refined_summary})
        messages.append({
            "role": "user",
            "content": "Create a final, polished version that captures the essence in a compelling way:"
        })
        
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.7
        )
        final_summary = response.choices[0].message.content
        
        self.print_box(
            "✨ Final Polished Summary",
            final_summary,
            "green"
        )
        
        print(f"✓ Completed analysis for: {filename}\n")
        return final_summary

    def process_files(self) -> None:
        """Process all files with reflection steps"""
        self.print_step("🤖 PHASE 2: PROCESSING AND ANALYZING FILES")
        
        for filename, content in self.files_content.items():
            self.print_step(f"📌 Processing File: {filename}")
            self.individual_summaries[filename] = self.reflect_and_summarize(
                content, filename
            )
            time.sleep(1)  # Avoid rate limiting

    def combine_summaries(self) -> str:
        """Combine all individual summaries into a final summary"""
        self.print_step("🎯 PHASE 3: GENERATING FINAL COMBINED SUMMARY")
        
        print("\n→ Collecting all individual summaries...")
        all_summaries = "\n\n".join([
            f"Summary of {filename}:\n{summary}" 
            for filename, summary in self.individual_summaries.items()
        ])
        
        print("\n→ Generating comprehensive final summary...")
        messages = [
            {"role": "system", "content": "You are a skilled document summarizer. Create a concise yet comprehensive combined summary that shows the relationships between all documents."},
            {"role": "user", "content": f"Create a unified summary that synthesizes these document summaries, highlighting key themes and connections:\n\n{all_summaries}"}
        ]
        
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.7
        )
        self.final_summary = response.choices[0].message.content
        
        self.print_box(
            "🌟 FINAL COMBINED SUMMARY OF ALL DOCUMENTS",
            self.final_summary,
            "purple"
        )
        
        print("\n✨ Summary process completed successfully! ✨")
        return self.final_summary

In [21]:
print("🚀 Starting Document Summarization Process...")
agent = SummarizationAgent()
agent.read_files()
agent.process_files()
agent.combine_summaries()

🚀 Starting Document Summarization Process...

📂 PHASE 1: READING FILES FROM DIRECTORY

→ Processing file: newsletter_one.txt


                                                                                                                   
   ╭─ 📄 File Content: newsletter_one.txt ─────────────────────────────────────────────────────────────────────╮   
   │                                                                                                           │   
   │   Link :                                                                                                  │   
   │   https://www.forbes.com/councils/forbestechcouncil/2025/01/02/ai-artificial-intelligence-or-artificial   │   
   │   -identity                                                                                               │   
   │                                                                                                           │   
   │                                                                                                           │   
   │   AI is widely recognized as an acronym for "artificial intelligence

                                                                                                                   
   ╭─ 📄 File Content: newsletter_three.txt ───────────────────────────────────────────────────────────────────╮   
   │                                                                                                           │   
   │   Link : https://blog.samaltman.com/reflections                                                           │   
   │                                                                                                           │   
   │                                                                                                           │   
   │   The second birthday of ChatGPT was only a little over a month ago, and now we have transitioned into    │   
   │   the next paradigm of models that can do complex reasoning. New years get people in a reflective mood,   │   
   │   and I wanted to share some personal thoughts about how it has gone

                                                                                                                   
   ╭─ 📄 File Content: newsletter_two.txt ─────────────────────────────────────────────────────────────────────╮   
   │                                                                                                           │   
   │   Link :                                                                                                  │   
   │   https://moderndiplomacy.eu/2024/12/30/ethical-artificial-intelligence-assessing-challenges-and-opport   │   
   │   unities                                                                                                 │   
   │                                                                                                           │   
   │                                                                                                           │   
   │   Ethical artificial intelligence (AI) is a binding point of discuss

                                                                                                                   
   ╭─ 📝 Initial Analysis Results ─────────────────────────────────────────────────────────────────────────────╮   
   │                                                                                                           │   
   │   Key Points Identified:                                                                                  │   
   │   The text discusses the evolving landscape of artificial intelligence (AI) and its potential             │   
   │   transformation into artificial identities. It highlights the broad scope of AI applications, ranging    │   
   │   from generative AI to deepfake technology, and the need for a clear definition and legislation to       │   
   │   regulate its usage. The text explores the concept of AI developing an identity, drawing parallels       │   
   │   between human and machine identities and questioning when AI can e

...

                                                                                                                   
   ╭─ 📝 Initial Analysis Results ─────────────────────────────────────────────────────────────────────────────╮   
   │                                                                                                           │   
   │   Key Points Identified:                                                                                  │   
   │   The text discusses the importance of ethical artificial intelligence (AI) and the challenges and        │   
   │   opportunities associated with its development and deployment. Key points include the following:         │   
   │                                                                                                           │   
   │   - Ethical AI involves using moral principles and ethical frameworks in the development, deployment,     │   
   │   and usage of AI techniques, with fundamental notions such as trans

                                                                                                                   
   ╭─ 🔄 Refined Analysis ─────────────────────────────────────────────────────────────────────────────────────╮   
   │                                                                                                           │   
   │   Structured Summary:                                                                                     │   
   │   The text highlights the significance of ethical artificial intelligence (AI) and the challenges and     │   
   │   opportunities it presents:                                                                              │   
   │                                                                                                           │   
   │   - Ethical AI involves using moral principles like transparency, responsibility, fairness, privacy,      │   
   │   and security in AI development.                                   

                                                                                                                   
   ╭─ ✨ Final Polished Summary ───────────────────────────────────────────────────────────────────────────────╮   
   │                                                                                                           │   
   │   The text explores the critical importance of ethical artificial intelligence (AI), emphasizing the      │   
   │   need for moral principles such as transparency, responsibility, fairness, privacy, and security in AI   │   
   │   development. Key challenges include privacy, accountability, bias, transparency, data bias, and         │   
   │   explainability. However, ethical AI also offers opportunities for improved decision-making,             │   
   │   innovation, and positive social impact. By integrating ethical considerations into AI practices and     │   
   │   adopting a multidisciplinary approach involving policymakers, deve

                                                                                                                   
   ╭─ 🌟 FINAL COMBINED SUMMARY OF ALL DOCUMENTS ──────────────────────────────────────────────────────────────╮   
   │                                                                                                           │   
   │   The documents collectively delve into the transformative journey of artificial intelligence (AI),       │   
   │   emphasizing the evolving landscape where AI may acquire identities and blur human-machine               │   
   │   interactions. They stress the importance of clear definitions, regulations, and ethical principles      │   
   │   like transparency, responsibility, fairness, and privacy to govern AI technologies. Discussions on      │   
   │   sentience, intellectual property dissemination, and fact-checking in AI-generated content are           │   
   │   explored, with a focus on adapting identity management tools to na

"The documents collectively delve into the transformative journey of artificial intelligence (AI), emphasizing the evolving landscape where AI may acquire identities and blur human-machine interactions. They stress the importance of clear definitions, regulations, and ethical principles like transparency, responsibility, fairness, and privacy to govern AI technologies. Discussions on sentience, intellectual property dissemination, and fact-checking in AI-generated content are explored, with a focus on adapting identity management tools to navigate this evolving space. OpenAI's pursuit of AGI and societal impact, alongside challenges in diverse leadership viewpoints and governance, highlights the complex nature of AI development. The integration of ethical considerations into AI practices, involving diverse stakeholders, policymakers, and ethicists, presents opportunities for improved decision-making, innovation, and positive social impact, while emphasizing the need for a harmonious ba